In [1]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
from helper import singleglob, Waveform, json_merge

In [2]:
base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Test_Julien_ForcedInput/")

# base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01")
dat_path = singleglob(base, "*.dat")
task_path = singleglob(base, "*.xls")
info_path = singleglob(base, "*.yaml", search_upward_limit=Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files"))
res_events_path = base/"events.tsv"
exists = {"dat_path":dat_path.exists(), "task_path":task_path.exists(), "info_path": info_path.exists()}
if not np.all(list(exists.values())):
    display(exists)
    raise Exception("Missing some input files...")
info_path

PosixPath('/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/rat_model.yaml')

In [3]:
event_df = pd.read_csv(dat_path, sep="\t", names=['time (ms)', 'family', 'nbre', '_P', '_V', '_L', '_R', '_T', '_W', '_X', '_Y', '_Z'], skiprows=13, dtype=int)
event_df.insert(0, "t", event_df.pop("time (ms)")/1000)
event_df = event_df.reset_index(names="poly_evnum").sort_values(["t", "poly_evnum"]).reset_index(drop=True)
event_df["task_node"] = event_df["_T"].where(event_df["family"]==10).ffill()
print(event_df.to_string())

     poly_evnum        t  family  nbre  _P  _V  _L  _R  _T   _W  _X   _Y  _Z  task_node
0             0    0.007      10     1   1   1   0   0   1    0   0    0   0        1.0
1             1    0.022      10     1   1   1   0   0   2    0   0    0   0        2.0
2             2    0.023       1     4   0   0   0   0   0    0   0    0   0        2.0
3             3   14.534       5     1   1   0   1   1   1    0   0    0   0        2.0
4             4   19.892       5     1   0   1   1   0   1    1   0    0   1        2.0
5             5   19.892       5     1   0   1   1   0   1    1   0    0   1        2.0
6             6   21.589       5     1   0   0   1   0   1   11   0   10   1        2.0
7             7   22.733       5     1   0   1   1   0   1   12   0   11   1        2.0
8             8   26.444       5     1   0   0   1   0   1   35   0   34   1        2.0
9             9   28.556       5     1   0   1   1   0   1   36   0   35   1        2.0
10           10   32.797       5

In [4]:
task_df = pd.read_csv(task_path, sep="\t", header=11)
task_df = task_df.rename(columns={task_df.columns[0]: "task_node" })
display(task_df.columns)
task_df

Index(['task_node', 'T1', 'T2', 'T3', 'T4', 'LED1(1,2)', 'LED2(1,3)',
       'LED3(1,4)', 'L1(2,1)', 'L2(2,2)', 'LK1(5,1)', 'LK2(5,2)', 'RD(6,11)',
       'ASND(6,20)', 'PAD(6,22)', 'STR1(13,1)', 'STR2(13,2)', 'TTLP1(15,1)',
       'TTLP2(15,2)', 'TTLP3(15,3)', 'TTLP4(15,4)', 'TTLP5(15,5)',
       'TTLP6(15,6)', 'TTLP7(15,7)', 'TTLP8(15,8)', 'NEXT1', 'NEXT2', 'NEXT3',
       'CT1', 'CT2'],
      dtype='object')

,task_node,T1,T2,T3,T4,"LED1(1,2)","LED2(1,3)","LED3(1,4)","L1(2,1)","L2(2,2)",...,"TTLP4(15,4)","TTLP5(15,5)","TTLP6(15,6)","TTLP7(15,7)","TTLP8(15,8)",NEXT1,NEXT2,NEXT3,CT1,CT2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,!ti,NaN,NaN,NaN,NaN
2,2.0,_essais,NaN,NaN,NaN,NaN,NaN,!on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,PAD_L=1,L1_L=1(16),L2_L=1(22),NaN,NaN
3,3.0,400-1200,_ITI,NaN,NaN,NaN,NaN,on,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,!ti,PAD_L=0(2),NaN,NaN,NaN
4,4.0,0-20,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,!TJ.TI,!TJ.!TI(8),PAD_L=0(2),NaN,NaN
5,5.0,2000_RT,NaN,NaN,NaN,on(50),NaN,!on,NaN,NaN,...,NaN,"ON(100,1,1)",NaN,NaN,NaN,PAD_P=0.PAD_V=1.ti,!TI(12),PAD_V=0.ti(20),CTT+1,NaN
6,6.0,3500_MT,_RT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,L1_L=1.PAD_P=0.PAD_V=1.ti,!TI(17),L2_L=1(14),NaN,NaN
7,7.0,300,_MT1,NaN,NaN,!on,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,LK1_P=0.!ti.LK1_V=0(11),NaN,NaN,CTX+1,NaN
8,8.0,2000_RT,NaN,NaN,NaN,NaN,on(50),!on,NaN,NaN,...,NaN,"ON(50,50,2)",NaN,NaN,NaN,PAD_V=0.PAD_P=1.ti,!TI(15),PAD_P=0.ti(21),CTT+1,NaN
9,9.0,3500_MT,_RT2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,L2_L=1.PAD_V=0.PAD_P=1.ti,!TI(19),L1_L=1(18),NaN,NaN


In [5]:


channels = pd.Series(task_df.columns).str.extract(r'\s*(?P<channel_name>\w+)\s*\((?P<family>\d+)\s*,\s*(?P<nbre>\d+)\)\s*').assign(taskcol_name=task_df.columns).dropna(how="any")
channels["family"] = channels["family"].astype(int)
channels["nbre"] = channels["nbre"].astype(int)
channels

    

,channel_name,family,nbre,taskcol_name
5,LED1,1,2,"LED1(1,2)"
6,LED2,1,3,"LED2(1,3)"
7,LED3,1,4,"LED3(1,4)"
8,L1,2,1,"L1(2,1)"
9,L2,2,2,"L2(2,2)"
10,LK1,5,1,"LK1(5,1)"
11,LK2,5,2,"LK2(5,2)"
12,RD,6,11,"RD(6,11)"
13,ASND,6,20,"ASND(6,20)"
14,PAD,6,22,"PAD(6,22)"


In [6]:
pattern=r'on\(\d+(,\d+)*\)'
task_info=pd.DataFrame()
stacked = task_df.set_index("task_node")[channels["taskcol_name"].to_list()].stack().str.lower().str.strip().dropna()
stacked.index.names=["task_node", "taskcol_name"]
task_info["data"] = stacked
task_info["match"] = task_info["data"].str.fullmatch(pattern)
task_info = task_info.loc[task_info["match"]]
task_info["important"] = task_info["data"].str.slice(3, -1)
task_info["task_params"] = task_info["important"].str.split(",").apply(lambda l: [float(x) for x in l])
task_info = task_info.drop(columns=["important", "match", "data"]).join(stacked.rename("task_data"), how="outer")
task_info = task_info.reset_index() 
task_info["task_node"] = task_info["task_node"].astype(float)
task_info


,task_node,taskcol_name,task_params,task_data
0,2.0,"LED3(1,4)",NaN,!on
1,3.0,"LED3(1,4)",NaN,on
2,5.0,"ASND(6,20)","[50.0, 1.0, 1.0, 30.0, 1000.0]","on(50,1,1,30,1000)"
3,5.0,"LED1(1,2)",[50.0],on(50)
4,5.0,"LED3(1,4)",NaN,!on
5,5.0,"TTLP1(15,1)","[100.0, 1.0, 1.0]","on(100,1,1)"
6,5.0,"TTLP3(15,3)","[100.0, 1.0, 1.0]","on(100,1,1)"
7,5.0,"TTLP5(15,5)","[100.0, 1.0, 1.0]","on(100,1,1)"
8,7.0,"LED1(1,2)",NaN,!on
9,7.0,"LK1(5,1)",[1.0],on(1)


In [7]:
event_channels_df = channels.merge(event_df, on=["family", "nbre"], how="right").merge(task_info, on=["taskcol_name", "task_node"], how="left").sort_values("t")
print(event_channels_df.to_string())

    channel_name  family  nbre taskcol_name  poly_evnum        t  _P  _V  _L  _R  _T   _W  _X   _Y  _Z  task_node                     task_params           task_data
0            NaN      10     1          NaN           0    0.007   1   1   0   0   1    0   0    0   0        1.0                             NaN                 NaN
1            NaN      10     1          NaN           1    0.022   1   1   0   0   2    0   0    0   0        2.0                             NaN                 NaN
2           LED3       1     4    LED3(1,4)           2    0.023   0   0   0   0   0    0   0    0   0        2.0                             NaN                 !on
3            LK1       5     1     LK1(5,1)           3   14.534   1   0   1   1   1    0   0    0   0        2.0                             NaN                 NaN
4            LK1       5     1     LK1(5,1)           4   19.892   0   1   1   0   1    1   0    0   1        2.0                             NaN                 NaN
5   

In [8]:
info = yaml.safe_load(info_path.open("r"))
info

{'processing': [{'duplicate_over': {'channel_name': ['LED(\\d+)',
     'ASND',
     'TTLP(1|3|4|8)']},
   'event_name': '{channel_name}',
   'method': 'output_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': '_P',
    'count_expr': '_L',
    'expected_count_expr': 'task[2].fillna(1)',
    'duration_on_expr': 'task[0]',
    'duration_off_expr': 'task[1].fillna(0)/1000'}},
  {'duplicate_over': {'channel_name': ['TTLP(2|5|6|7)']},
   'event_name': '{channel_name}',
   'method': 'input_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': '_P'}},
  {'duplicate_over': {'channel_name': ['L(\\d+)']},
   'event_name': '{channel_name}',
   'method': 'input_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': '_V'}},
  {'duplicate_over': {'side': ['_P', '_V']},
   'event_name': 'PAD{side}',
   'method': 'input_binary_wave',
   'method_params': {'fi

In [9]:
event_spec = []

for item in info["processing"]:
    if "duplicate_over" not in item:
        item["duplicate_over"] = {}
    from helper import generate_duplicate_table, replace_vals
    duplication = generate_duplicate_table(item["duplicate_over"], dict(channel_name=channels["channel_name"]))
    for _, row in duplication.iterrows():
        final_d = replace_vals(item, row.to_dict())
        ev_name = final_d["event_name"]
        if "display" in info and "rename"in info["display"] and ev_name in info["display"]["rename"]:
            final_d["display_name"] = info["display"]["rename"][ev_name]
        else:
            final_d["display_name"] = ev_name
        del final_d["duplicate_over"]
        event_spec.append(final_d)

unique_df = pd.DataFrame(event_spec)["event_name"].value_counts().reset_index()
if not (unique_df["count"] == 1).all():
    display(unique_df.loc[unique_df["count"] > 1])
    raise Exception(f"Event name duplication")


display(pd.DataFrame(event_spec))
event_spec = {v["event_name"]: v for v in event_spec}


,event_name,method,method_params,display_name,metadata
0,LED1,output_binary_wave,"{'filter_expr': 'channel_name==""LED1""', 'state...",LED1,NaN
1,LED2,output_binary_wave,"{'filter_expr': 'channel_name==""LED2""', 'state...",LED2,NaN
2,LED3,output_binary_wave,"{'filter_expr': 'channel_name==""LED3""', 'state...",LED3,NaN
3,ASND,output_binary_wave,"{'filter_expr': 'channel_name==""ASND""', 'state...",ASND,NaN
4,TTLP1,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP1""', 'stat...",cue,NaN
5,TTLP3,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP3""', 'stat...",TTLP3,NaN
6,TTLP4,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP4""', 'stat...",TTLP4,NaN
7,TTLP8,output_binary_wave,"{'filter_expr': 'channel_name==""TTLP8""', 'stat...",TTLP8,NaN
8,TTLP2,input_binary_wave,"{'filter_expr': 'channel_name==""TTLP2""', 'stat...",TTLP2,NaN
9,TTLP5,input_binary_wave,"{'filter_expr': 'channel_name==""TTLP5""', 'stat...",TTLP5,NaN


In [10]:
event_channels_df = event_channels_df.sort_values(["t", "poly_evnum"])
event_channels_df["used"]=False
from ipywidgets import widgets
outs = {k: display(display_id=f'{k}') for k in event_spec}
def compute_relevant(config):
    df = pd.DataFrame()
    ev_name = config["event_name"]
    df["t"] = event_channels_df["t"]
    # df["task_params"] = event_channels_df["task_params"]
    df["task_data"] = event_channels_df["task_data"]
    df["task_node"] = event_channels_df["task_node"]
    task_expr_pattern = re.compile(r'task\[(?P<col_num>\d+)\]')
    for param, expr in config["method_params"].items():
        if param.endswith("_expr"):
            task_expr_df = pd.DataFrame()
            def handle_col(match):
                num = int(match["col_num"])
                name = f'__task_{num}'
                task_expr_df[name] = event_channels_df["task_params"].apply(lambda l: l[num] if isinstance(l, list) and len(l) > num else np.nan)
                return name
            new_expr=re.sub(task_expr_pattern, handle_col, expr)
            df[param.replace("_expr", "_value")] = pd.concat([event_channels_df, task_expr_df], axis=1).eval(new_expr)
        else:
            df[param] = expr
    if "metadata" in config:
        metadata = pd.DataFrame()
        for k, v in config["metadata"].items():
            metadata[k] = event_channels_df.eval(v)
        df["metadata"] = metadata.apply(lambda row: row.to_dict(), axis=1)
    else:
        df["metadata"] = [{}] * len(df.index)
    filtered = df.loc[df["filter_value"]] if "filter_expr" in config["method_params"] else df
    if "state_expr" in config["method_params"]:
        relevant = filtered.loc[filtered["state_value"] != filtered["state_value"].shift(1)].copy()
        relevant["duration"] = relevant["t"].shift(-1) - relevant["t"]
    else:
        relevant = filtered.copy()
    outs[config["event_name"]].update(relevant.rename_axis([ev_name]))
    return relevant



,t,task_data,task_node,filter_value,state_value,count_value,expected_count_value,duration_on_value,duration_off_value,metadata,duration
LED1,,,,,,,,,,,
28,91.455,on,22.0,True,1,1,1.0,NaN,0.0,{},5.999
33,97.454,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.177
40,98.631,on,22.0,True,1,1,1.0,NaN,0.0,{},6.000
45,104.631,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.216
52,105.847,on,16.0,True,1,1,1.0,NaN,0.0,{},5.999
65,111.846,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.119
74,112.965,on,16.0,True,1,1,1.0,NaN,0.0,{},6.000
79,118.965,!on,13.0,True,0,0,1.0,NaN,0.0,{},45.332
113,164.297,on(50),5.0,True,1,1,1.0,50.0,0.0,{},0.050


,t,task_data,task_node,filter_value,state_value,count_value,expected_count_value,duration_on_value,duration_off_value,metadata,duration
LED2,,,,,,,,,,,
27,91.455,on,22.0,True,1,1,1.0,NaN,0.0,{},5.999
32,97.454,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.177
39,98.631,on,22.0,True,1,1,1.0,NaN,0.0,{},6.000
44,104.631,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.216
51,105.847,on,16.0,True,1,1,1.0,NaN,0.0,{},5.999
64,111.846,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.119
73,112.965,on,16.0,True,1,1,1.0,NaN,0.0,{},6.000
78,118.965,!on,13.0,True,0,0,1.0,NaN,0.0,{},47.325
121,166.290,on,12.0,True,1,1,1.0,NaN,0.0,{},6.000


,t,task_data,task_node,filter_value,state_value,count_value,expected_count_value,duration_on_value,duration_off_value,metadata,duration
LED3,,,,,,,,,,,
2,0.023,!on,2.0,True,0,0,1.0,NaN,0.0,{},91.432
26,91.455,on,22.0,True,1,1,1.0,NaN,0.0,{},5.999
31,97.454,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.177
38,98.631,on,22.0,True,1,1,1.0,NaN,0.0,{},6.000
43,104.631,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.216
50,105.847,on,16.0,True,1,1,1.0,NaN,0.0,{},5.999
63,111.846,!on,13.0,True,0,0,1.0,NaN,0.0,{},1.119
72,112.965,on,16.0,True,1,1,1.0,NaN,0.0,{},6.000
77,118.965,!on,13.0,True,0,0,1.0,NaN,0.0,{},43.360


,t,task_data,task_node,filter_value,state_value,count_value,expected_count_value,duration_on_value,duration_off_value,metadata,duration
ASND,,,,,,,,,,,
111,164.297,"on(50,1,1,30,1000)",5.0,True,1,1,1.0,50.0,0.001,{},0.050
114,164.347,"on(50,1,1,30,1000)",5.0,True,0,1,1.0,50.0,0.001,{},8.908
138,173.255,"on(50,1,1,30,5000)",8.0,True,1,1,1.0,50.0,0.001,{},0.049
141,173.304,"on(50,1,1,30,5000)",8.0,True,0,1,1.0,50.0,0.001,{},9.588
163,182.892,"on(50,1,1,30,5000)",8.0,True,1,1,1.0,50.0,0.001,{},0.050
166,182.942,"on(50,1,1,30,5000)",8.0,True,0,1,1.0,50.0,0.001,{},10.325
192,193.267,"on(50,1,1,30,1000)",5.0,True,1,1,1.0,50.0,0.001,{},0.050
195,193.317,"on(50,1,1,30,1000)",5.0,True,0,1,1.0,50.0,0.001,{},29.809
220,223.126,"on(50,1,1,30,1000)",5.0,True,1,1,1.0,50.0,0.001,{},0.050


,t,task_data,task_node,filter_value,state_value,count_value,expected_count_value,duration_on_value,duration_off_value,metadata,duration
TTLP1,,,,,,,,,,,
110,164.297,"on(100,1,1)",5.0,True,1,1,1.0,100.0,0.001,{},0.095
118,164.392,"on(100,1,1)",5.0,True,0,1,1.0,100.0,0.001,{},8.863
137,173.255,"on(100,1,1)",8.0,True,1,1,1.0,100.0,0.001,{},0.100
143,173.355,"on(100,1,1)",8.0,True,0,1,1.0,100.0,0.001,{},9.537
162,182.892,"on(100,1,1)",8.0,True,1,1,1.0,100.0,0.001,{},0.101
168,182.993,"on(100,1,1)",8.0,True,0,1,1.0,100.0,0.001,{},10.274
191,193.267,"on(100,1,1)",5.0,True,1,1,1.0,100.0,0.001,{},0.101
199,193.368,"on(100,1,1)",5.0,True,0,1,1.0,100.0,0.001,{},29.758
219,223.126,"on(100,1,1)",5.0,True,1,1,1.0,100.0,0.001,{},0.101


...

,t,task_data,task_node,filter_value,state_value,duration_on_value,expected_count_value,metadata,duration
LK2_reward,,,,,,,,,


,t,task_data,task_node,filter_value,state_value,metadata,duration
polytask_linechange,,,,,,,
0,0.007,NaN,1.0,True,1,{},0.015
1,0.022,NaN,2.0,True,2,{},68.712
21,68.734,NaN,1.0,True,1,{},22.720
25,91.454,NaN,22.0,True,22,{},6.000
30,97.454,NaN,13.0,True,13,{},0.500
...,...,...,...,...,...,...,...
279,274.442,NaN,5.0,True,5,{},0.257
292,274.699,NaN,6.0,True,6,{},3.500
294,278.199,NaN,17.0,True,17,{},6.000


,t,task_data,task_node,filter_value,state_value,metadata,duration
zone_change,,,,,,,
23,86.176,NaN,1.0,True,0,{},45.171
82,131.347,NaN,2.0,True,1,{},10.931
92,142.278,NaN,2.0,True,0,{},NaN


,t,task_data,task_node,filter_value,metadata
RD,,,,,


,t,task_data,task_node,filter_value,metadata
pause,,,,,
304,291.188,NaN,2.0,True,{'cause': 1}


In [11]:

def compute_metadata(join_metadata, **warnings):
    kept_warnings = {}
    for k, v in warnings.items():
        vals = {v for v in v.values() if not pd.isna(v)}
        
        if len(vals) > 1:
            kept_warnings[k] = v
    if len(kept_warnings) > 0:
        metadata = json_merge(*join_metadata, dict(warnings=kept_warnings))
    else:
        metadata = json_merge(*join_metadata)
    return metadata


def output_accumulator_binary_wave(relevant: pd.DataFrame, config):
    res = []
    for _,grp in  relevant.groupby((relevant["state_value"] ==1).cumsum()):
        starts = grp[grp["state_value"]==1]
        if len(starts.index) ==0: continue
        elif len(starts.index) > 1: display(grp); raise Exception(f"Problem {len(starts.index)}")
        else:
            start = starts["t"].iat[0]
            rises = [0] + (grp["t"].loc[grp["state_value"] > grp["state_value"].shift(1)] -start).to_list()
            duration_on = grp["duration_on_value"].iat[0]
            expected_count = grp["expected_count_value"].iat[0]
            duration = rises[-1] + duration_on
            metadata = compute_metadata(grp["metadata"], count_mismatch=dict(read=len(rises), expected=expected_count))
            res.append(dict(t=start, metadata=metadata, duration=duration, waveform_info=
                dict(type="binary", rises=rises, durations=[duration_on]*len(rises), change_speed=0.0001)))
    return pd.DataFrame(res)

def input_binary_wave(relevant: pd.DataFrame, config):
    if relevant["state_value"].iat[0] == 0:
        relevant = relevant.iloc[1:, :].copy()
    relevant["next_metadata"] = relevant["metadata"].shift(-1)
    rises = relevant.iloc[::2].copy()
    rises["metadata"] = rises.apply(lambda row: compute_metadata(
        [row["metadata"], row["next_metadata"]]), axis=1)
    return rises[["t", "duration", "metadata"]].assign(waveform_info=rises["duration"].apply(lambda d: 
        dict(type="binary", rises=[0], durations=[d], base_value=0, event_value=1, change_speed=0.0001)))
    

def output_binary_wave(relevant: pd.DataFrame, config):
    if relevant["state_value"].iat[0] == 0:
        relevant = relevant.iloc[1:, :].copy()
    relevant["next_metadata"] = relevant["metadata"].shift(-1)
    relevant["count_value"] = relevant["count_value"].shift(-1).replace(0,1)
    rises = relevant.iloc[::2].copy()

    rises["read_duration"] = rises["duration"]
    rises["duration"] = np.where(
        (rises["duration"] <= 0.001) & rises["duration_on_value"].notna(), 
        rises["duration_on_value"], rises["duration"])
    
    # rises["duration"] = rises["duration"].fillna(-1)
    rises["cycle_duration"] = rises["duration_on_value"] + rises["duration_off_value"]
    rises["rises"] = rises.apply(lambda row: [i * row["cycle_duration"] for i in range(int(row["count_value"]))] if not pd.isna(row["count_value"]) else np.nan, axis=1)
    # rises["duration"] = np.where(rises["duration"]>=0, rises["duration"], np.nan)
    # display(rises)
    rises["metadata"] = rises.apply(lambda row: compute_metadata(
        [row["metadata"], row["next_metadata"]], 
        count_mismatch=dict(read=row["count_value"], expected=row["expected_count_value"]),
        duration_correction=dict(read=row["read_duration"], corrected=row["duration"] if row["duration"] >=0 else np.nan)
    ), axis=1)
    
    rises["waveform_info"] = rises.apply(lambda row: 
        dict(type="binary", rises=row["rises"], durations=[row["duration_on_value"]]* len(row["rises"]), base_value=0, event_value=1, change_speed=0.0001), axis=1)
    return rises[["t","metadata", "duration", "waveform_info"]].copy()



In [12]:



all = []
for ev_name, item in event_spec.items():
    relevant = compute_relevant(item)
    if len(relevant.index) ==0:
        continue
    if item["method"] == "output_accumulator_binary_wave":
        events = output_accumulator_binary_wave(relevant, item)
    elif item["method"] == "input_binary_wave":
        events = input_binary_wave(relevant, item)
    elif item["method"] == "output_binary_wave":
        events = output_binary_wave(relevant, item)
    elif item["method"]=="step_wave":
        events = relevant.assign(curr_value= relevant["state_value"], prev_value=relevant["state_value"].shift(1), next_value=relevant["state_value"].shift(-1))
        events["waveform_info"] = events[["curr_value", "prev_value", "next_value"]].apply(lambda row: row.to_dict(), axis=1)
        events = events[["t","metadata", "duration", "waveform_info"]]
    elif item["method"]=="event":
        events = relevant[["t", "metadata"]].assign(duration=np.nan, waveform_info=None)
    if len(events.index)!=0:
        all.append(events[[c for c in events.columns if events[c].count() > 0]].assign(event_name=ev_name))
all = pd.concat(all).sort_values("t")
display(all)
  

,t,metadata,duration,waveform_info,event_name
0,0.007,{},0.015,"{'curr_value': 1.0, 'prev_value': nan, 'next_v...",polytask_linechange
1,0.022,{},68.712,"{'curr_value': 2.0, 'prev_value': 1.0, 'next_v...",polytask_linechange
0,14.534,{},0.200,"{'type': 'binary', 'rises': [0], 'durations': ...",LK1_reward
4,19.892,{},1.697,"{'type': 'binary', 'rises': [0], 'durations': ...",LK1_lick
7,22.733,{},3.711,"{'type': 'binary', 'rises': [0], 'durations': ...",LK1_lick
...,...,...,...,...,...
295,278.199,{},6.000,"{'type': 'binary', 'rises': [nan], 'durations'...",LED3
294,278.199,{},6.000,"{'curr_value': 17.0, 'prev_value': 6.0, 'next_...",polytask_linechange
298,284.199,{},0.500,"{'curr_value': 13.0, 'prev_value': 17.0, 'next...",polytask_linechange
302,284.699,{},NaN,"{'curr_value': 2.0, 'prev_value': 13.0, 'next_...",polytask_linechange


In [13]:
if "display" in info and "rename" in info["display"]:
    all["event_name"] = all["event_name"].map(lambda e: info["display"]["rename"][e] if e in info["display"]["rename"] else e)

In [15]:

all["metadata_json"] = all["metadata"].apply(lambda d: json.dumps(d) if not pd.isna(d) else "{}")
all["waveform_info_json"] = all["waveform_info"].apply(lambda d: json.dumps(d) if not pd.isna(d) else "{}")

all.drop(columns=["waveform_info", "metadata"]).to_csv(res_events_path, sep="\t", index=False)

In [16]:
reloaded = pd.read_csv(res_events_path, sep="\t", index_col=False)
reloaded["metadata"] = reloaded["metadata_json"].apply(lambda s: json.loads(s))
reloaded["waveform_info"] = reloaded["waveform_info_json"].apply(lambda s: json.loads(s))
reloaded.drop(columns=["waveform_info_json", "metadata_json"])

,t,duration,event_name,metadata,waveform_info
0,0.007,0.015,polytask_linechange,{},"{'curr_value': 1.0, 'prev_value': nan, 'next_v..."
1,0.022,68.712,polytask_linechange,{},"{'curr_value': 2.0, 'prev_value': 1.0, 'next_v..."
2,14.534,0.200,LK1_reward,{},"{'type': 'binary', 'rises': [0], 'durations': ..."
3,19.892,1.697,LK1_lick,{},"{'type': 'binary', 'rises': [0], 'durations': ..."
4,22.733,3.711,LK1_lick,{},"{'type': 'binary', 'rises': [0], 'durations': ..."
...,...,...,...,...,...
169,278.199,6.000,LED3,{},"{'type': 'binary', 'rises': [nan], 'durations'..."
170,278.199,6.000,polytask_linechange,{},"{'curr_value': 17.0, 'prev_value': 6.0, 'next_..."
171,284.199,0.500,polytask_linechange,{},"{'curr_value': 13.0, 'prev_value': 17.0, 'next..."
172,284.699,NaN,polytask_linechange,{},"{'curr_value': 2.0, 'prev_value': 13.0, 'next_..."
